In [ ]:
import os
import pandas as pd
from glob import glob
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [ ]:
# Dataset folders
# video_size_2019-01-26T11_2019-01-27T11
dataset = 'video_sent_2019-01-26T11_2019-01-27T11'
buffer_dataset = 'client_buffer_2019-01-26T11_2019-01-27T11'

current_dir = os.getcwd()
base_path = '' + current_dir + '/puffer_tests/'

# Load video_sent data
file_path = os.path.join(base_path + dataset + '.csv')    
try:
    data = pd.read_csv(file_path, dtype=str)
    print(f"Loaded {file_path} with shape {data.shape}")
    data['source_dataset'] = dataset
except Exception as e:
    print(f"Error reading {file_path}: {e}")

# Load client_buffer data
file_path = os.path.join(base_path + buffer_dataset + '.csv')
try:
    buffer_data = pd.read_csv(file_path, dtype=str)
    print(f"Loaded {file_path} with shape {buffer_data.shape}")
    buffer_data['source_dataset'] = buffer_dataset
except Exception as e:
    print(f"Error reading {file_path}: {e}")

In [ ]:
# Ensure the time column is numeric
data['time (ns GMT)'] = pd.to_numeric(data['time (ns GMT)'], errors='coerce')
buffer_data['time (ns GMT)'] = pd.to_numeric(buffer_data['time (ns GMT)'], errors='coerce')

# Drop rows with null time values (from coercion or original data)
data = data.dropna(subset=['time (ns GMT)', 'session_id'])
buffer_data = buffer_data.dropna(subset=['time (ns GMT)', 'session_id'])

# Sort both dataframes properly
data_sorted = data.sort_values(['time (ns GMT)', 'session_id']).reset_index(drop=True)
buffer_sorted = buffer_data.sort_values(['time (ns GMT)', 'session_id']).reset_index(drop=True)

In [ ]:
# Merge asof (requires full sort on the merge key)
merged = pd.merge_asof(
    data_sorted,
    buffer_sorted,
    on='time (ns GMT)',
    by='session_id',
    direction='backward',
    tolerance=10**9 * 10  # 10 seconds
)

print(merged)

In [ ]:
expt_settings = []
with open('expt_settings', 'r') as f:
    for line in f:
        # Skip line number if present
        line = line.strip()
        if not line:
            continue
        # Split off the line number if present (e.g., "1 {json}")
        parts = line.split(' ', 1)
        if len(parts) == 2:
            _, json_str = parts
        else:
            json_str = parts[0]
        obj = json.loads(json_str)
        expt_settings.append(obj)

In [ ]:
data = merged
print(data.columns)

In [ ]:
def split_dfs(data):
    # Split the dataframe based on unique values in the 'category' column
    return {group: group_df for group, group_df in data.groupby('expt_id')}

def split_dfs_merged(data):
    # Split the dataframe based on unique values in the 'category' column
    return {group: group_df for group, group_df in data.groupby('expt_id_x')}

split_dataframes = split_dfs_merged(data)

# Create base output directory
base_dir = "ABR_Separated"

# Make sure the base directory exists
os.makedirs(base_dir, exist_ok=True)

# Dictionary to keep count of files saved per ABR
abr_counts = {}
# Now split_dfs is a dictionary where the key is the category, and the value is the corresponding DataFrame
for key, sub_df in split_dataframes.items():
    abr_name = expt_settings[int(key)-1]["abr"]
    abr_dir = os.path.join(base_dir, abr_name)

    # Make sure the ABR-specific directory exists
    os.makedirs(abr_dir, exist_ok=True)

    # Determine filename
    count = abr_counts.get(abr_name, 0) + 1
    abr_counts[abr_name] = count
    filename = f"{count}.csv"

    # Full path to save
    file_path = os.path.join(abr_dir, filename)
    sub_df.to_csv(file_path, index=False)

    print(f"Saved: {file_path}")